Dataset is from [movielens data set](https://grouplens.org/datasets/movielens/). 
Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares (ALS). The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

## 1. Import and get spark instance

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [2]:
spark = SparkSession.builder.appName('recomender system').getOrCreate()

## 2. Explore dataset

In [3]:
df = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
df.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [5]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

## 3. Model and Train

In [6]:
(training, test) = df.randomSplit([0.8, 0.2])  # 80% and 20%

#### Model and train

In [7]:
%%time
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

CPU times: user 4.35 ms, sys: 9.22 ms, total: 13.6 ms
Wall time: 8.42 s


#### Prediction

In [8]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [9]:
predictions.show()  # 'prediction' is rating prediction

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   4.0|    12| 0.76345867|
|     31|   1.0|    19| 0.70779073|
|     31|   1.0|     4|   2.703626|
|     85|   1.0|    26|  1.0535085|
|     85|   3.0|     6| 0.53574574|
|     85|   1.0|     4|-0.42131245|
|     85|   4.0|     7| -1.4615539|
|     65|   1.0|     4|  0.5755451|
|     53|   1.0|    12| -1.8776163|
|     53|   1.0|    23|-0.35032412|
|     78|   1.0|    28|  1.1643512|
|     78|   1.0|    17|  0.6217128|
|     78|   1.0|    24|  0.4460914|
|     34|   1.0|    17|-0.10235441|
|     34|   1.0|     0|  0.6973809|
|     81|   1.0|     1|  1.9570845|
|     81|   1.0|     6|   4.116249|
|     28|   1.0|     6|  0.1782273|
|     28|   1.0|    10|  1.4647185|
|     76|   1.0|    20|  0.3694229|
+-------+------+------+-----------+
only showing top 20 rows



#### Evaluation

In [10]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))  # rmse ~1.7 in a rating value from 1 to 5 so it is not good

Root-mean-square error = 1.6775960584294478


The RMSE described our error in terms of the stars rating column.

#### Example of "rating" prediction for a new user 

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [11]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
single_user.show() # User had 10 ratings in the test data set 
                   # Realistically this should be some sort of hold out set!

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     12|    11|
|     20|    11|
|     22|    11|
|     23|    11|
|     25|    11|
|     30|    11|
|     36|    11|
|     38|    11|
|     48|    11|
|     59|    11|
+-------+------+



In [12]:
reccomendations = model.transform(single_user)

In [13]:
reccomendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     20|    11| 1.8813366|
|     30|    11| 1.7943391|
|     12|    11| 1.7040048|
|     48|    11| 1.4515016|
|      6|    11| 1.4234176|
|     22|    11| 0.7660456|
|     59|    11| 0.7300942|
|     36|    11| 0.7232655|
|     23|    11|  0.521365|
|     38|    11|0.39585602|
|     25|    11|0.10891601|
+-------+------+----------+

